In [1]:
#This is the script to test the neural network:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#We operate on the CPU beacuse we can't do better for the moment
device = torch.device("cpu")

In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #Input = Multiphase flow properties Output =  neurons
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 5)
    
    def forward(self, x):
        #Activation function in F.relu (rectified linear)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc4(x)
        
        return F.log_softmax(x, dim=1, )

In [4]:
net = Net().to(device)
checkpoint = torch.load('./NNKLD.pth', map_location='cpu')
net.load_state_dict(checkpoint['net_state_dict'])
net.eval()

Net(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=5, bias=True)
)

In [5]:
#Here one should load the dataset:
df_test = pd.read_csv('Data/Test.csv')
df_test.head()

,P,T,DenL,DenG,VisL,ST,ID,Ang,Vsl,Vsg,Flow Pattern.1
0,103.5,25.0,1000.0,1.200,0.001,0.070,0.0510,-80.0,0.898129,0.243692,3
1,100.0,20.0,1000.0,1.120,0.001,0.070,0.0254,0.0,0.121951,4.481707,4
2,100.0,22.0,860.0,4.134,0.007,0.032,0.0512,-1.0,0.371803,0.839364,4
3,103.9,25.0,1000.0,1.200,0.001,0.070,0.0250,-80.0,0.106229,4.068800,2
4,103.1,25.0,1000.0,1.200,0.001,0.070,0.0250,-30.0,0.010311,0.015488,2


In [6]:
#Here we can test different performance of the neural netowrk with respect to smaller subset of the test dataset
df_test.reset_index(drop=True, inplace=True)

In [7]:
col_names = df_test.iloc[:,:-1].columns

In [8]:
#This dataset is raw, we can however we have the normalizing constants from the preprocessing script
normalizing = pd.read_csv('Data/Normalizing.csv')
#We take every element of the df and normalize them:
val_n = np.zeros([df_test.shape[0], df_test.shape[1]-1])
for isig in range(df_test.shape[1]-1):
    val_n[:,isig]=(df_test.iloc[:,isig]-normalizing.iloc[isig,0])/normalizing.iloc[isig,1]
    
df_test = pd.concat([pd.DataFrame(val_n, columns=col_names), df_test.iloc[:,-1]], axis=1)

In [9]:
#We select the total dataset to test its accuracy
X_old = np.array(df_test.iloc[:,:-1]) #Features
Y_old = np.array(df_test.iloc[:,-1])  #Labels

X_validation = torch.Tensor(X_old)
Y_validation = torch.Tensor(np.eye(5)[[i for i in Y_old]])

accuracy = []
cc = []
cr = []

errors = []
labels = []
predicted = []

for j in range(5):
    acc = 0
    count = 0
    for i in range(0, len(X_validation)):  
        if torch.argmax(Y_validation[[i]]) == j:
            net_out = net(X_validation[[i]])
            if torch.argmax(net_out).item() == torch.argmax(Y_validation[[i]]):
                acc +=1
                count +=1
            else:
                count +=1
                errors.append(X_old[i]) #We store where we commit the error in the error list
                predicted.append(torch.argmax(net_out).item())
                labels.append(Y_old[i]) #We store the missed label in the label list
                
    if count==0:
        accuracy.append(np.nan)
    else:
        accuracy.append(acc/count)
        
    cr.append(acc)                    
    cc.append(count)
    
print(accuracy, cc) #Kinda a cross entropy matrix

[0.8673469387755102, 0.8189655172413793, 0.9234234234234234, 0.8910891089108911, 0.9150326797385621] [196, 116, 222, 303, 306]


In [10]:
print(sum(cr)/sum(cc))

0.8923884514435696


In [11]:
#Test single elements of the neural network:
net_out = net(X_validation[[0]])
Dict = {0: 'Dispersed Bubble', 1: 'Stratifeid Smoth', 2: 'Stratified Wavy', 3:'Annular', 4:'Intermittent'} 
Dict[torch.argmax(net_out).item()]

'Annular'

In [12]:
Dict = {0: 'DB', 1: 'SS', 2: 'SW', 3:'AN', 4:'IN'} 
y_pred = []
y_true = []
for i in range(0, len(X_validation)):
    net_out = net(X_validation[[i]])
    y_pred.append(Dict[torch.argmax(net_out).item()])
    y_true.append(Dict[torch.argmax(Y_validation[[i]]).item()])
    
y_true = pd.Series(np.array(y_true))
y_pred = pd.Series(np.array(y_pred))
cross = pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

cross

Predicted,AN,DB,IN,SS,SW,All
True,,,,,,
AN,270,0,18,4,11,303
DB,0,170,26,0,0,196
IN,5,10,280,4,7,306
SS,5,0,12,95,4,116
SW,8,0,8,1,205,222
All,288,180,344,104,227,1143


In [13]:
try:
    df_errors = pd.DataFrame(errors)
    df_label = pd.DataFrame(labels, columns=['True'])
    df_predicted = pd.DataFrame(predicted, columns=['Predicted'])
    #We remove the normalization:
    val_n = np.zeros([df_errors.shape[0], df_errors.shape[1]])
    for isig in range(df_errors.shape[1]):
        val_n[:,isig]=(df_errors.iloc[:,isig]*normalizing.iloc[isig,1])+normalizing.iloc[isig,0]

    df_errors = pd.concat([pd.DataFrame(val_n, columns=col_names), df_label, df_predicted], axis=1)


    df_errors.to_csv('Data/Errors.csv', index=False)
except:
    print('Congratulation, no error found!')